In [2]:
import pandas as pd
pd.options.display.max_rows=100
df_out=pd.read_csv("heart.csv")
df_out.isnull().sum()
df_out.head()
df_out["Zscore"]=(df_out.Cholesterol-df_out["Cholesterol"].mean())/df_out["Cholesterol"].std()
df_out.head()
df_out=df_out[(df_out["Zscore"]<3) | (df_out["Zscore"]>-3)]
df_out


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Zscore
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,0.824621
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.171867
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,0.769768
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0.138964
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,-0.034736
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,0.596068
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,-0.053020
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,-0.619830
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0.340090


In [38]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
encoder=LabelEncoder()
df_out["Sex"]=encoder.fit_transform(df_out.Sex)
df_out["ExerciseAngina"]=encoder.fit_transform(df_out.ExerciseAngina)
# df_out["ChestPainType"]=encoder.fit_transform(df_out.ChestPainType)
# df_out["RestingECG"]=encoder.fit_transform(df_out.RestingECG)
encoder1=OneHotEncoder(sparse_output=False)
ct=["ChestPainType","RestingECG","ST_Slope"]
df1=encoder1.fit_transform(df_out[ct])
df2=pd.DataFrame(df1,columns=encoder1.get_feature_names_out(ct))
# df_out.drop(ct,axis=1,inplace=True)
df3=pd.concat([df_out,df2],axis=1)
# df_out.drop(["ChestPainType","ST_Slope","RestingECG"],axis=1)
# df_out






NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [52]:
# df3.drop("Zscore",axis=1,inplace=True)
x=df3.drop(["HeartDisease","ChestPainType_ASY","RestingECG_LVH","ST_Slope_Down"],axis=1)
y=df3["HeartDisease"]
x.shape

(918, 15)

In [26]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
arr=scaler.fit_transform(x)
arr

array([[-1.4331398 ,  0.51595242,  0.41090889, ..., -0.49044933,
        -1.00218103,  1.15067399],
       [-0.47848359, -1.93816322,  1.49175234, ..., -0.49044933,
         0.99782372, -0.86905588],
       [-1.75135854,  0.51595242, -0.12951283, ...,  2.03894663,
        -1.00218103,  1.15067399],
       ...,
       [ 0.37009972,  0.51595242, -0.12951283, ..., -0.49044933,
         0.99782372, -0.86905588],
       [ 0.37009972, -1.93816322, -0.12951283, ..., -0.49044933,
         0.99782372, -0.86905588],
       [-1.64528563,  0.51595242,  0.30282455, ..., -0.49044933,
        -1.00218103,  1.15067399]])

In [40]:
from sklearn.model_selection import train_test_split
x_train,y_train,x_val,y_val=train_test_split(arr,y,test_size=0.2,random_state=42)

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model=LogisticRegression()
model.fit(x_train,x_val)
y_predict=model.predict(y_train)
model1=SVC()
model1.fit(x_train,x_val)
y1_predict=model1.predict(y_train)
y_predict


array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1])

In [67]:
from sklearn.metrics import accuracy_score
# result=accuracy_score(y_val,y_predict)
# result1=accuracy_score(y_val,y1_predict)
# print(result)
print(model.score(x_train,x_val))
# result1
print(model1.score(x_train,x_val))

0.8719346049046321
0.9100817438692098


In [113]:
from sklearn.decomposition import PCA
pca=PCA(0.95)
x_pca=pca.fit_transform(x)
pca.explained_variance_ratio_
# pca.n_components_
print(x_pca)

[[ 92.31131546  29.44565085]
 [-17.14366463  13.73602879]
 [ 81.90671738 -38.22351056]
 ...
 [-69.00377342 -17.33460169]
 [ 39.20676111  33.59195039]
 [-21.4375636   37.21660999]]


In [107]:
x1=x_pca
x_train,y_train,x_val,y_val=train_test_split(x1,y,test_size=0.2,random_state=42)
print(x_train)

[[ 4.43086557e+01  5.73444714e+01 -1.14750064e-01 -1.47535086e+00
   6.44053057e-01]
 [ 1.26676377e+01  4.23896521e+01  5.89800632e+00 -1.10640213e+01
  -3.60232632e-01]
 [ 1.37725958e+01 -1.65458965e+01  1.38455549e+01 -1.54330290e+00
   1.73065525e-01]
 ...
 [ 5.44183306e+01  3.47830700e+00 -1.52755624e+00  7.84088879e+00
   2.50475024e-01]
 [-1.99235895e+02 -1.31141845e+01  2.08292969e+01  5.37872369e-01
  -1.15438115e+00]
 [ 1.92869745e+02 -1.85154385e+01  8.15641217e+00 -1.60375001e+01
   1.16002772e+00]]


In [114]:
model={
    "LogisticRegression":LogisticRegression(max_iter=1000),
    "SVC":SVC(),
   
}
for model in model.values():
    # print(x_train)
    model.fit(x_train,x_val)
    model.predict(y_train)
    print(model.score(x_train,x_val))

0.8147138964577657
0.7438692098092643


In [115]:
model1=SVC()
model.fit(x_train,x_val)
y_predict=model.predict(y_train)
model.score(x_train,x_val)


0.7438692098092643